# Naive Bayes for Sentiment Analysis

We will be using [this](http://ai.stanford.edu/~amaas/data/sentiment/) dataset for binary sentiment classification. The dataset contains 25,000 highly polar movie reviews for training, and 25,000 for testing. To get the data:

`wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz`

## Load Data

In [1]:
from pyspark import SparkContext 
import numpy as np
from collections import Counter

In [2]:
sc = SparkContext.getOrCreate()
path = "/Users/Yimei/Desktop/winter/spark-nb/aclImdb/"
train_path = path + "train/"
test_path = path + "test/"

In [3]:
data_raw_pos = sc.textFile(train_path + "pos/*.txt")
data_raw_neg = sc.textFile(train_path + "neg/*.txt")

In [4]:
# note that this is a whole review
data_raw_pos.first()

u'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [5]:
# sample 20% of the data
data_raw_pos = data_raw_pos.sample(False, 0.2, 1)
data_raw_neg = data_raw_neg.sample(False, 0.2, 1)

In [6]:
# number of partitions
data_raw_pos.getNumPartitions()

12500

In [7]:
# You may OR may NOT want to repartition or coalesce
# num_partitions = 3 or 4 times the number of CPUs
num_partitions = 8
data_raw_pos = data_raw_pos.repartition(num_partitions)
data_raw_neg = data_raw_neg.repartition(num_partitions)

In [8]:
# count 2529 elements
# this takes some time as the count is doing the action
print(data_raw_pos.count())
print(data_raw_neg.count())

2529
2529


## Training NB

In [9]:
# split into words (here we could filter stopwords, clean, romove punctuation)
data_pos = data_raw_pos.flatMap(lambda x: x.split())
data_pos.take(10)
#want the whole bag of words thus flatmap it 

[u'A',
 u'lot',
 u'of',
 u'people',
 u'are',
 u'saying',
 u'that',
 u'Al',
 u'Pacino',
 u'over']

In [10]:
# transform to value pairs to be able to count
data_pos = data_pos.map(lambda x: (x, 1))
data_pos.take(10)

[(u'A', 1),
 (u'lot', 1),
 (u'of', 1),
 (u'people', 1),
 (u'are', 1),
 (u'saying', 1),
 (u'that', 1),
 (u'Al', 1),
 (u'Pacino', 1),
 (u'over', 1)]

In [11]:
# counting number of words
data_pos = data_pos.reduceByKey(lambda x,y:x+y)
data_pos.take(10)

[(u'remastered', 1),
 (u'gangs.', 1),
 (u'anyways.I', 1),
 (u'wonderfull', 1),
 (u'four', 68),
 (u'Francisco,', 4),
 (u'demotes', 1),
 (u'HE-MAN', 1),
 (u'(sometimes', 2),
 (u'electricity', 5)]

In [12]:
# we can do all together
data_neg = data_raw_neg.flatMap(lambda x: x.split()).map(lambda x: (x, 1)).reduceByKey(lambda x,y:x+y)
data_neg.take(10)

[(u'Jojo', 2),
 (u'/>of', 3),
 (u"syberberg's", 1),
 (u'lungs.<br', 1),
 (u'Montford.', 1),
 (u'increase', 2),
 (u'jobbing', 1),
 (u'electricity', 1),
 (u"'Breakfast", 3),
 (u're-united', 1)]

How should we compute count(pos) and count(neg)?

In [13]:
count_pos = data_pos.map(lambda x: x[1]).reduce(lambda x,y:x+y)
count_neg = data_neg.map(lambda x: x[1]).reduce(lambda x,y:x+y)

In [14]:
print(count_pos, count_neg)
#here the number of the two clases are the same, half and half so we can simply ignore the log multiplication of that part

(604442, 572063)


In [15]:
## Let's get V
v1 = data_pos.map(lambda x: x[0]) # pos vocabulary
v2 = data_neg.map(lambda x: x[0]) # neg vocabulary
v = v1.union(v2)
#v.count()
v0 = v.distinct()
V = v0.count()
print(V)

101820


In [16]:
# Note that the denominators are different 
pos_denom = float(count_pos + V + 1)
neg_denom = float(count_neg + V + 1)

In [17]:
# log probabities
pos_prob = data_pos.map(lambda x: (x[0], np.log(float(x[1] + 1)/pos_denom)))

neg_prob = data_neg.map(lambda x: (x[0], np.log(float(x[1] + 1)/neg_denom))) 

In [18]:
pos_prob.take(10)

[(u'remastered', -12.77459578779308),
 (u'gangs.', -12.77459578779308),
 (u'anyways.I', -12.77459578779308),
 (u'wonderfull', -12.77459578779308),
 (u'four', -9.2336364637557669),
 (u'Francisco,', -11.858305055918926),
 (u'demotes', -12.77459578779308),
 (u'HE-MAN', -12.77459578779308),
 (u'(sometimes', -12.369130679684917),
 (u'electricity', -11.675983499124971)]

In [19]:
pos_prob = dict(pos_prob.collect())
neg_prob = dict(neg_prob.collect())

#change into a dictionary, easier to use to look up when throw in test set 

In [20]:
# broadcast = shared by all nodes
pos_prob_b = sc.broadcast(pos_prob)
neg_prob_b = sc.broadcast(neg_prob)

## Prediction

In [21]:
test_raw_pos = sc.textFile(test_path + "pos/*.txt")
test_raw_neg = sc.textFile(test_path + "neg/*.txt")

test_raw_pos = test_raw_pos.sample(False, 0.2, 1)
test_raw_neg = test_raw_neg.sample(False, 0.2, 1)

num_partitions = 8
test_raw_pos = test_raw_pos.repartition(num_partitions)
test_raw_neg = test_raw_neg.repartition(num_partitions)

print(test_raw_pos.count())
print(test_raw_neg.count())

2529
2529


In [22]:
doc = test_raw_pos.first()
print(doc)

Why this film was only released in 4 states is beyond me. I thought this film was a divine story. The name says it all: Seeing Other People. This movie has more logic than laughs, which I suppose is why it works so well. Common sense also makes an appearance in what would seem to be another puerile sex comedy. Alice is getting her feet frozen in the cold, when she feels irrationally about the way she might perform for her fiancé, not just sexually, but as a partner, and friend etc. This starts what seems to be an almost archetypal journey for the both of them. One fling after another leads to trouble, as if it wasn't a bad idea from the start. Witty dialogue and comic set-ups make this one funny as hell! Nicholson and Mohr set the tone of the film early on, and keep the promise they anticipate. Other highlights are Lauren Graham, Andy Richter, and Helen Slater(in her first theatrical film in 10 years!). Climax begins to take an insane turn, but a simple ending makes this one far more e

In [23]:
def pred_class(doc):
    words = doc.split(" ")
    counts = Counter(words)
    log_pos = 0.0
    log_neg = 0.0
    for w in counts:
        log_pos += counts[w]* pos_prob_b.value.get(w, np.log(1.0/pos_denom))
        log_neg += counts[w]* neg_prob_b.value.get(w, np.log(1.0/neg_denom))
    if log_pos > log_neg:
        return "pos"
    return "neg"

In [24]:
pred_class(doc)

'pos'

In [25]:
test_pos_res = test_raw_pos.map(pred_class)
test_pos_res.take(10)

['pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'neg', 'neg', 'pos', 'pos']

In [26]:
test_pos_res = test_raw_pos.map(pred_class).map(lambda x: (x, 1)).reduceByKey(lambda x,y:x+y)
pos_results = dict(test_pos_res.collect())
print(pos_results)

{'neg': 575, 'pos': 1954}


In [27]:
test_neg_res = test_raw_neg.map(pred_class).map(lambda x: (x, 1)).reduceByKey(lambda x,y:x+y)
neg_results = dict(test_neg_res.collect())
print(neg_results)

{'neg': 2153, 'pos': 376}


In [28]:
# compute accuracy
total = sum(neg_results.values()) + sum(pos_results.values())
acc = float(neg_results["neg"] + pos_results["pos"]) / float(total)
print(acc)

0.811981020166
